In [4]:
import pandas as pd
from xml.etree import ElementTree
from xml.etree.ElementTree import Element

import os


MERCANTE = 'mercante'

In [5]:
lista_arquivos = os.listdir(MERCANTE)

In [241]:
print(lista_arquivos[0])

MERCANTE_CONHECIMENTO_2019-09-27-16-00-00.xml


In [229]:
with open(os.path.join(MERCANTE, lista_arquivos[1]), 'r', encoding='iso-8859-1') as file1:
    content1 = file1.read()

In [230]:
xtree = ElementTree.parse(os.path.join(MERCANTE, lista_arquivos[1]))
xroot = xtree.getroot()

In [231]:
print(xroot)
for node in xroot:
    print(node)
    print(node.tag)
    if isinstance(node, Element):
        for subnode in node:
            print(subnode.tag)
            print(subnode.text)


<Element 'cargas' at 0x7f0bf933fea8>
<Element 'itensCarga' at 0x7f0bf933f458>
itensCarga
tipoMovimento
A
dataAtualizacao
2019-09-27
horaAtualizacao
15:10:24
numeroCEmercante
151905216436912
tipoItemCarga
1
numeroSequencialItemCarga
0001
pesoBruto
000006220.000
codigoConteiner
MSCU4459376
isoCode
42P3
tara
000005.800
indicadorUsoParcial
false
cubagemM3
0000000060.000
lacre


NCM


<Element 'itensCarga' at 0x7f0bf8fb6728>
itensCarga
tipoMovimento
A
dataAtualizacao
2019-09-27
horaAtualizacao
15:10:24
numeroCEmercante
151905216436912
tipoItemCarga
1
numeroSequencialItemCarga
0002
pesoBruto
000009360.000
codigoConteiner
MSCU4466591
isoCode
42P2
tara
000005.350
indicadorUsoParcial
false
cubagemM3
0000000060.000
lacre


NCM


<Element 'itensCarga' at 0x7f0bf8fb6ea8>
itensCarga
tipoMovimento
I
dataAtualizacao
2019-09-27
horaAtualizacao
15:10:38
numeroCEmercante
181905216443406
tipoItemCarga
1
numeroSequencialItemCarga
0001
pesoBruto
000056139.600
codigoConteiner
TGBU7058381
isoCode
45G0
tara
0

000027110.000
codigoConteiner
TRHU1364454
isoCode
22G0
tara
002180.000
indicadorUsoParcial
false
cubagemM3
0000000028.000
lacre


NCM


<Element 'itensCarga' at 0x7f0bf9098098>
itensCarga
tipoMovimento
I
dataAtualizacao
2019-09-27
horaAtualizacao
15:12:15
numeroCEmercante
011907216444890
tipoItemCarga
1
numeroSequencialItemCarga
0001
pesoBruto
000006166.060
codigoConteiner
APHU6743231
isoCode
45G0
tara
003870.000
indicadorUsoParcial
false
cubagemM3
0000000072.020
lacre


NCM


<Element 'itensCarga' at 0x7f0bf90985e8>
itensCarga
tipoMovimento
I
dataAtualizacao
2019-09-27
horaAtualizacao
15:12:15
numeroCEmercante
011907216444970
tipoItemCarga
1
numeroSequencialItemCarga
0001
pesoBruto
000003357.000
codigoConteiner
AFIU5806088
isoCode
22T0
tara
003357.000
indicadorUsoParcial
false
cubagemM3
0000000012.000
lacre


lacre


NCM


<Element 'itensCarga' at 0x7f0bf9098bd8>
itensCarga
tipoMovimento
I
dataAtualizacao
2019-09-27
horaAtualizacao
15:12:15
numeroCEmercante
011907216445004
tipoItemCar

tipoMovimento
I
dataAtualizacao
2019-09-27
horaAtualizacao
15:17:51
numeroCEmercante
151905216449739
tipoItemCarga
1
numeroSequencialItemCarga
0001
pesoBruto
000023250.000
codigoConteiner
MNBU0299940
isoCode
45R1
tara
004260.000
indicadorUsoParcial
false
cubagemM3
0000000050.000
lacre


NCM


<Element 'itensCarga' at 0x7f0bf910c318>
itensCarga
tipoMovimento
I
dataAtualizacao
2019-09-27
horaAtualizacao
15:18:25
numeroCEmercante
151905216449810
tipoItemCarga
1
numeroSequencialItemCarga
0001
pesoBruto
000017384.200
codigoConteiner
TCNU4632198
isoCode
45G0
tara
003900.000
indicadorUsoParcial
false
cubagemM3
0000000035.280
lacre


NCM


NCM


NCM


NCM


<Element 'itensCarga' at 0x7f0bf910ca48>
itensCarga
tipoMovimento
I
dataAtualizacao
2019-09-27
horaAtualizacao
15:18:26
numeroCEmercante
151905216449909
tipoItemCarga
1
numeroSequencialItemCarga
0001
pesoBruto
000022340.000
codigoConteiner
MSCU5546217
isoCode
42G0
tara
003740.000
cubagemM3
0000000047.000
lacre


NCM


<Element 'itensCarga' 

dataAtualizacao
2019-09-27
horaAtualizacao
15:14:16
numeroCEmercante
161905207837656
tipoItemCarga
3
numeroSequencialItemCarga
0001
pesoBruto
000579985.000
cubagemM3
0000000686.536
codigoTipoGranel
05
descricaoGranel
ULTRA LOW SULPHUR DIESEL 10 PPM NCM: 2710.19.21
NCM


<Element 'itensCarga' at 0x7f0bf8d52138>
itensCarga
tipoMovimento
A
dataAtualizacao
2019-09-27
horaAtualizacao
15:16:14
numeroCEmercante
161905208002280
tipoItemCarga
3
numeroSequencialItemCarga
0001
pesoBruto
002073122.000
cubagemM3
0000002500.000
codigoTipoGranel
05
descricaoGranel
DIESEL 500PPM NCM: 2710.19.21
NCM


<Element 'itensCarga' at 0x7f0bf8d52bd8>
itensCarga
tipoMovimento
A
dataAtualizacao
2019-09-27
horaAtualizacao
15:16:27
numeroCEmercante
161907216437748
tipoItemCarga
1
numeroSequencialItemCarga
0001
pesoBruto
000027086.400
codigoConteiner
BMOU2409140
isoCode
22G1
tara
002220.000
indicadorUsoParcial
false
cubagemM3
0000000032.166
lacre


NCM


<Element 'itensCarga' at 0x7f0bf8d6b3b8>
itensCarga
tipoMovime

In [216]:
class ParseXML:

    def _campos(self):
        return [campo for campo in dir(self) if campo[0] != '_']

    def _parse_node(self, node):
        for campo in self._campos():
            # print(campo)
            alvo = node.find(campo) 
            if alvo is not None:
                # print(alvo.text, alvo.tag)
                destino = getattr(self, campo)
                if isinstance (destino, str):
                    setattr(self, campo, alvo.text)
                else:
                    setattr(self, campo, alvo)
            else:
                print('Não encontrou %s' % campo)

    def _to_dict(self):
        result = {}
        for campo in self._campos():
            valor = getattr(self, campo)
            if isinstance(valor, ParseXML):
                valor = str(valor)
            result[campo] = valor 
        return result
        

class Embarcador(ParseXML):
    cnpjShipper = ''
    idEmbarcador = ''

    def __repr__(self):
        return self.cnpjShipper
    
class Consignatario(ParseXML):
    tipoConsignatario = ''
    cnpjConsignatario = ''
    dadosComplementaresConsignatario = ''

    def __repr__(self):
        return self.cnpjConsignatario

class Conhecimento(ParseXML):
    def __init__(self):
        self.tipoMovimento:str = ''
        self.dataAtualizacao:str = ''
        self.horaAtualizacao:str = ''
        self.tipoTrafego:str = ''
        self.tipoBLConhecimento:str = ''
        self.numeroCEMaster:str = ''
        self.dataEmissao:str = ''
        self.cubagem:str = ''
        self.portoDestFinal:str = ''
        self.portoOrigemCarga:str = ''
        self.descricao:str = ''
        self.numeroCEmercante:str = ''
        self._embarcador:Element = None
        self._consignatario:Element = None

    
    @property
    def embarcador(self):
        return self._embarcador
        
    @embarcador.setter
    def embarcador(self, node: Element):
        self._embarcador = Embarcador()
        self._embarcador._parse_node(node)

    @property
    def consignatario(self):
        return self._consignatario
        
    @consignatario.setter
    def consignatario(self, node: Element):
        self._consignatario = Consignatario()
        self._consignatario._parse_node(node)

        
    # def __repr__(self):
    #    repr = ' '.join([ campo + '-' + getattr(self, campo) for campo in self._campos()])
    #    if repr is None:
    #        return 'ERRO'
    #    return repr
        

manifestosCarga
<tipoMovimento>A</tipoMovimento>
<numero>1919301835474</numero>
<codAgenciaInformante>02427026000146</codAgenciaInformante>
<codigoEmpresaNavegacao>02427026000146</codigoEmpresaNavegacao>
<numeroImoDPC>9625396</numeroImoDPC>
<dataEncerramento>2019-09-13</dataEncerramento>
<dataInicioOperacao>2019-09-27</dataInicioOperacao>
<portoCarregamento>BRSSA</portoCarregamento>
<portoDescarregamento>BRIBB</portoDescarregamento>
<tipoTrafego>3</tipoTrafego>
<numeroViagem>937S</numeroViagem>
<quantidadeConhecimento>7</quantidadeConhecimento>
<dataAtualizacao>2019-09-27</dataAtualizacao>
<horaAtualizacao>00:01:18</horaAtualizacao>
<codigoTerminalCarregamento>BRSSA002</codigoTerminalCarregamento>
<codigoTerminalDescarregamento>BRIBB002</codigoTerminalDescarregamento>
<numeroEscalaMercante>19000279154</numeroEscalaMercante><numeroEscalaMercante>19000279162</numeroEscalaMercante><numeroEscalaMercante>19000279170</numeroEscalaMercante><numeroEscalaMercante>19000279189</numeroEscalaMercante><numeroEscalaMercante>19000336905</numeroEscalaMercante>

In [217]:
Conhecimento()._campos()

['consignatario',
 'cubagem',
 'dataAtualizacao',
 'dataEmissao',
 'descricao',
 'embarcador',
 'horaAtualizacao',
 'numeroCEMaster',
 'numeroCEmercante',
 'portoDestFinal',
 'portoOrigemCarga',
 'tipoBLConhecimento',
 'tipoMovimento',
 'tipoTrafego']

In [232]:
classes = {'conhecimentosEmbarque': Conhecimento}
    
    
objetos = []
for node in xroot:
    classe = classes.get(node.tag)
    if classe:
        # print(node.tag, node.text)
        objeto = classe()
        objeto._parse_node(node)
        # objeto = Conhecimento()
        # objeto._parse_node(node)
        # print(objeto)
        objetos.append(objeto._to_dict())
        
        # break

In [233]:
[(campo, getattr(objeto, campo)) for campo in objeto._campos()]

[('consignatario', None),
 ('cubagem', '0000000135.000'),
 ('dataAtualizacao', '2019-09-27'),
 ('dataEmissao', '2019-09-23'),
 ('descricao',
  '1 CONTAINER WITH                        ARGENTINE KABULI CHICK PEAS             GRADE 2 CROP 2019                       1 CONTAINER WITH                        ARGENTINE KABULI CHICK PEAS             GRADE 2 CROP 2019                       1 CONTAINER WITH                        ARGENTINE KABULI CHICK PEAS             GRADE 2 CROP 2019                       1 CONTAINER WITH                        ARGENTINE KABULI CHICK PEAS             GRADE 2 CROP 2019                       1 CONTAINER WITH'),
 ('embarcador', ),
 ('horaAtualizacao', '15:59:21'),
 ('numeroCEMaster', ''),
 ('numeroCEmercante', '001909034537706'),
 ('portoDestFinal', 'INNIP'),
 ('portoOrigemCarga', 'ARZAE'),
 ('tipoBLConhecimento', '10'),
 ('tipoMovimento', 'A'),
 ('tipoTrafego', '9')]

In [234]:
objeto.embarcador

In [235]:
objeto.embarcador.cnpjShipper

''

In [236]:
df_conhecimentos = pd.DataFrame(objetos)

In [237]:
df_conhecimentos.head()

""


In [238]:
df_conhecimentos.shape

(0, 0)

In [239]:
len(df_conhecimentos.numeroCEmercante.value_counts())

AttributeError: 'DataFrame' object has no attribute 'numeroCEmercante'

In [226]:
df_conhecimentos.tipoMovimento.value_counts()

I    397
A    107
E      3
Name: tipoMovimento, dtype: int64

In [227]:
!pip install mysql-connector-python

In [228]:
# !pip install sqlalchemy
import sqlalchemy

